In [1]:
!nvidia-smi

Wed Oct 25 19:05:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
import os
import time
import numpy as np
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
npy_files_path = "/content/drive/MyDrive/WCEBleedGen/Implementation"
os.chdir(npy_files_path)
import utils
import models

## Data Load

In [ ]:
# Define the paths to the training data
## train_data_path = "/content/drive/MyDrive/WCEBleedGen/TrainData"
## X_train, X_val, y_train, y_val = utils.load_data(train_data_path, save = False)

In [7]:
## npy_files_path = "/content/drive/MyDrive/WCEBleedGen/Implementation"
X_train = np.load('X_train.npy')
X_val = np.load('X_val.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')

In [8]:
# Print the shapes of the data and labels
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

X_train shape: (2094, 224, 224, 3)
y_train shape: (2094,)
X_val shape: (524, 224, 224, 3)
y_val shape: (524,)


## Model Build

In [19]:
model1 = models.create_model1(image_shape = (224, 224, 3))

In [20]:
# Compile the model
model1.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001,
                                                    weight_decay = 0.004,
                                                    use_ema = True,
                                                    ema_momentum = 0.9),
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics = ['accuracy'])

In [21]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 C1 (Conv2D)                 (None, 224, 224, 32)      2432      
                                                                 
 MP1 (MaxPooling2D)          (None, 112, 112, 32)      0         
                                                                 
 DropOut1 (Dropout)          (None, 112, 112, 32)      0         
                                                                 
 C2 (Conv2D)                 (None, 108, 108, 64)      51264     
                                                                 
 MP2 (MaxPooling2D)          (None, 54, 54, 64)        0         
                                                                 
 DropOut2 (Dropout)          (None, 54, 54, 64)        0         
                                                                 
 C3 (Conv2D)                 (None, 50, 50, 128)      

In [9]:
model2 = models.create_model2(input_shape = (224, 224, 3))

9406464/9406464 [==============================] - 0s 0us/step


In [12]:
# Compile the model
model2.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01),
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics = ['accuracy'])

In [13]:
model2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambd  (None, 224, 224, 3)       0         
 a)                                                              
                                                                 
 tf.math.subtract (TFOpLamb  (None, 224, 224, 3)       0         
 da)                                                             
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                             

In [14]:
model3 = models.create_resnet50(input_shape = (224, 224, 3), classes = 2)

In [15]:
# Compile the model
model3.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01),
              loss = tf.keras.losses.CategoricalCrossentropy,
              metrics = ['accuracy'])

In [16]:
model3.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_3[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 64)         9472      ['zero_padding2d[0][0]']      
                                                                                                  
 batch_normalization (Batch  (None, 112, 112, 64)         256       ['conv2d[0][0]']        

## Train

In [22]:
tick = time.perf_counter()
history1 = model1.fit(
    X_train, y_train,
    batch_size = 25,
    epochs = 50,
    verbose = 1,
    validation_data = (X_val, y_val),
)

tock = time.perf_counter()
print(f"Time to train model: {(tock-tick)/60:.2f} minutes.")

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5820: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


84/84 [==============================] - 15s 117ms/step - loss: 0.7356 - accuracy: 0.6953 - val_loss: 0.4914 - val_accuracy: 0.7595
Epoch 2/50
84/84 [==============================] - 7s 84ms/step - loss: 0.2179 - accuracy: 0.9155 - val_loss: 0.5906 - val_accuracy: 0.7004
Epoch 3/50
84/84 [==============================] - 7s 87ms/step - loss: 0.1515 - accuracy: 0.9475 - val_loss: 0.9395 - val_accuracy: 0.6355
Epoch 4/50
84/84 [==============================] - 7s 87ms/step - loss: 0.0938 - accuracy: 0.9661 - val_loss: 0.5568 - val_accuracy: 0.7939
Epoch 5/50
84/84 [==============================] - 7s 89ms/step - loss: 0.0558 - accuracy: 0.9799 - val_loss: 0.7156 - val_accuracy: 0.7195
Epoch 6/50
84/84 [==============================] - 7s 86ms/step - loss: 0.0539 - accuracy: 0.9814 - val_loss: 0.7861 - val_accuracy: 0.7118
Epoch 7/50
84/84 [==============================] - 7s 89ms/step - loss: 0.0345 - accuracy: 0.9881 - val_loss: 0.9682 - val_accuracy: 0.6336
Epoch 8/50
84/84 [====

In [17]:
tick = time.perf_counter()
history2 = model2.fit(
    X_train, y_train,
    batch_size = 25,
    epochs = 25,
    verbose = 1,
    validation_data = (X_val, y_val),
)

tock = time.perf_counter()
print(f"Time to train model: {(tock-tick)/60:.2f} minutes.")

Epoch 1/25
84/84 [==============================] - 19s 81ms/step - loss: 0.7842 - accuracy: 0.5024 - val_loss: 0.7484 - val_accuracy: 0.5057
Epoch 2/25
84/84 [==============================] - 5s 54ms/step - loss: 0.7623 - accuracy: 0.5119 - val_loss: 0.7145 - val_accuracy: 0.5000
Epoch 3/25
84/84 [==============================] - 5s 55ms/step - loss: 0.7271 - accuracy: 0.5172 - val_loss: 0.6700 - val_accuracy: 0.5000
Epoch 4/25
84/84 [==============================] - 5s 59ms/step - loss: 0.7173 - accuracy: 0.5310 - val_loss: 0.6953 - val_accuracy: 0.7271
Epoch 5/25
84/84 [==============================] - 5s 54ms/step - loss: 0.7117 - accuracy: 0.5263 - val_loss: 0.6591 - val_accuracy: 0.5000
Epoch 6/25
84/84 [==============================] - 4s 49ms/step - loss: 0.7109 - accuracy: 0.5435 - val_loss: 0.8504 - val_accuracy: 0.5000
Epoch 7/25
84/84 [==============================] - 5s 55ms/step - loss: 0.7246 - accuracy: 0.5478 - val_loss: 0.7205 - val_accuracy: 0.5000
Epoch 8/25
8

In [18]:
tick = time.perf_counter()
history3 = model3.fit(
    X_train, y_train,
    batch_size = 25,
    epochs = 50,
    verbose = 1,
    validation_data = (X_val, y_val),
)

tock = time.perf_counter()
print(f"Time to train model: {(tock-tick)/60:.2f} minutes.")

Epoch 1/50


TypeError: ignored